In [1]:
from pymongo import MongoClient

# Connect to MongoDB on localhost and the default port 27017
client = MongoClient('localhost', 27017)

# Create the database
db = client['steam_games_db']

# Check if the connection was successful
print("Databases:", client.list_database_names())


Databases: ['admin', 'config', 'local']


In [5]:
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['steam_games_db']

# Load games.csv into a pandas DataFrame
games_df = pd.read_csv(r'C:\Users\Dell E7440\Desktop\GAMES\games.csv')

# Convert DataFrame to a list of dictionaries (MongoDB documents)
games_data = games_df.to_dict(orient='records')

# Create the 'games' collection
games_collection = db['games']

# Insert data into the 'games' collection
games_collection.insert_many(games_data)

# Check the insertion by counting the documents
print("Inserted documents:", games_collection.count_documents({}))


Inserted documents: 101744


In [3]:
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['steam_games_db']

# Load users.csv into a pandas DataFrame
users_df = pd.read_csv(r'C:\Users\Dell E7440\Desktop\GAMES\users.csv')

# Convert DataFrame to a list of dictionaries (MongoDB documents)
users_data = users_df.to_dict(orient='records')

# Create the 'users' collection
users_collection = db['users']

# Insert data into the 'users' collection
users_collection.insert_many(users_data)

# Check the insertion by counting the documents
print("Inserted documents in users:", users_collection.count_documents({}))


Inserted documents in users: 28612128


In [2]:
#Recommendations file was causing errors so we will try to load it in chunks of 10000

import pandas as pd
from pymongo import MongoClient

# Reconnect to MongoDB
client = MongoClient('localhost', 27017)
db = client['steam_games_db']  

# Let the size of each chunk be 10000
chunk_size = 10000 

# Create or access the 'recommendations' collection
recommendations_collection = db['recommendations']

# Process the CSV file in chunks
for chunk in pd.read_csv(r'C:\Users\Dell E7440\Desktop\GAMES\recommendations.csv', chunksize=chunk_size):
    # Convert chunk to list of MongoDB docs
    recommendations_data = chunk.to_dict(orient='records')
    
    # Insert chunk into the MongoDB collection
    recommendations_collection.insert_many(recommendations_data)

    # Print progress
    print(f"Inserted {len(recommendations_data)} documents")

# Check the total number of documents inserted
print("Total documents in recommendations:", recommendations_collection.count_documents({}))


Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents
Inserted 10000 documents


In [5]:
#The last file in the GAMES folder is games_metadata in json format
import json
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['steam_games_db']

# Load games_metadata.json line by line
games_metadata_collection = db['games_metadata']

with open(r'C:\Users\Dell E7440\Desktop\GAMES\games_metadata.json', 'r', encoding='utf-8') as f:
    for line in f:
        try:
            # Parse each line as JSON
            game_metadata = json.loads(line.strip())
            # Insert into MongoDB
            games_metadata_collection.insert_one(game_metadata)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {e}")

# Verify the insertion
print("Total documents in games_metadata:", games_metadata_collection.count_documents({}))



Total documents in games_metadata: 50872


In [ ]:
#6 views in which a filtering is performed

In [7]:
#we will filter games by a specific tag (e.g., "Comedy")
db.create_collection(
    "games_metadata_comedy", 
    viewOn="games_metadata", 
    pipeline=[
        { "$match": { "tags": "Comedy" } }
    ]
)

print("View 'games_metadata_comedy' created.")








View 'games_metadata_comedy' created.


In [8]:
#we will filter games by a specific tag (e.g., "Adventure")

# Create a view to filter games with the tag "Adventure"
db.create_collection(
    "games_metadata_adventure", 
    viewOn="games_metadata", 
    pipeline=[
        { "$match": { "tags": "Adventure" } }
    ]
)

print("View 'games_metadata_adventure' created.")







View 'games_metadata_adventure' created.


In [9]:
#We will filter users with more than 50 reviews

db.create_collection(
    "users_with_many_reviews", 
    viewOn="users", 
    pipeline=[
        { "$match": { "reviews": { "$gt": 50 } } }
    ]
)

print("View 'users_with_many_reviews' created.")







View 'users_with_many_reviews' created.


In [11]:
#We are going to filter users with more than 100 products purchased
db.create_collection(
    "users_with_many_products", 
    viewOn="users", 
    pipeline=[
        { "$match": { "products": { "$gt": 50 } } }
    ]
)

print("View 'users_with_many_products' created.")
  


View 'users_with_many_products' created.


In [12]:
# We will create a view to filter games released after first January 2022
db.create_collection(
    "games_released_after_2022", 
    viewOn="games_metadata", 
    pipeline=[
        { "$match": { "date_release": { "$gt": "2022-01-01" } } } 
    ]
)

print("View 'games_released_after_2022' created.")

View 'games_released_after_2022' created.


In [19]:
#We will create a view to filter games reviewed more than 200 times
db.create_collection(
    "games_reviewed_more_than_200", 
    viewOn="recommendations", 
    pipeline=[
        { 
            "$group": { 
                "_id": "$app_id", 
                "review_count": { "$sum": 1 } 
            } 
        },  # Group by app_id and count reviews
        { 
            "$match": { 
                "review_count": { "$gt": 200 }  # Filter for review counts greater than 200
            } 
        },  
        { 
            "$lookup": {  # Lookup to join with the games_metadata collection to get game details
                "from": "games_metadata", 
                "localField": "_id", 
                "foreignField": "app_id", 
                "as": "game_info"
            }
        },
        { 
            "$unwind": "$game_info"  # Unwind the game_info array
        },  
        { 
            "$replaceRoot": { 
                "newRoot": { 
                    "$mergeObjects": [ 
                        "$game_info", 
                        { "review_count": "$review_count" } 
                    ] 
                } 
            }  
        }  
    ]  # Closing bracket for the pipeline array
)  # Closing parenthesis for create_collection

print("View 'games_reviewed_more_than_200' created.")





View 'games_reviewed_more_than_200' created.


In [15]:
#6 views in which an aggregation is performed


In [25]:
# Count the number of games by tag

db.games_metadata.aggregate([
    { "$unwind": "$tags" },  # Unwind the tags array
    { "$group": { "_id": "$tags", "count": { "$sum": 1 } } }  # Group by tags and count games
])






In [27]:
#Count the Average Rating per Genre
db.create_collection(
    "average_rating_per_genre", 
    viewOn="games_metadata", 
    pipeline=[
        { "$unwind": "$genres" },  # Unwind genres array
        { 
            "$group": { 
                "_id": "$genres", 
                "average_rating": { "$avg": "$rating" }  # Calculate average rating
            } 
        }
    ]
)

print("View 'average_rating_per_genre' created.")


View 'average_rating_per_genre' created.


In [28]:
#Count the Total Number of Reviews per Game
db.create_collection(
    "total_reviews_per_game", 
    viewOn="recommendations", 
    pipeline=[
        { 
            "$group": { 
                "_id": "$app_id", 
                "total_reviews": { "$sum": 1 }  # Count total reviews
            } 
        },
        { 
            "$lookup": { 
                "from": "games_metadata", 
                "localField": "_id", 
                "foreignField": "app_id", 
                "as": "game_info"
            } 
        },
        { "$unwind": "$game_info" },  # Unwind game_info array
        { 
            "$replaceRoot": { 
                "newRoot": { 
                    "$mergeObjects": [ 
                        "$game_info", 
                        { "total_reviews": "$total_reviews" } 
                    ] 
                } 
            } 
        }
    ]
)

print("View 'total_reviews_per_game' created.")








View 'total_reviews_per_game' created.


In [29]:
#Count the Average Playtime per Game

db.create_collection(
    "average_playtime_per_game", 
    viewOn="recommendations", 
    pipeline=[
        { 
            "$group": { 
                "_id": "$app_id", 
                "average_playtime": { "$avg": "$hours" }  # Calculate average playtime
            } 
        },
        { 
            "$lookup": { 
                "from": "games_metadata", 
                "localField": "_id", 
                "foreignField": "app_id", 
                "as": "game_info"
            } 
        },
        { "$unwind": "$game_info" },  # Unwind game_info array
        { 
            "$replaceRoot": { 
                "newRoot": { 
                    "$mergeObjects": [ 
                        "$game_info", 
                        { "average_playtime": "$average_playtime" } 
                    ] 
                } 
            } 
        }
    ]
)

print("View 'average_playtime_per_game' created.")






View 'average_playtime_per_game' created.


In [30]:
#Countthe average rating per Year of Release

db.create_collection(
    "average_rating_per_year", 
    viewOn="games_metadata", 
    pipeline=[
        { 
            "$group": { 
                "_id": { "$year": "$date_release" },  # Extract year from date_release
                "average_rating": { "$avg": "$rating" }  # Calculate average rating
            } 
        },
        { "$sort": { "_id": 1 } }  # Sort by year ascending
    ]
)

print("View 'average_rating_per_year' created.")







View 'average_rating_per_year' created.


In [32]:
 #Count the Average Rating by Platform

db.create_collection(
    "average_rating_by_platform", 
    viewOn="games_metadata", 
    pipeline=[
        { 
            "$project": { 
                "app_id": 1, 
                "rating": 1, 
                "platforms": { 
                    "$setUnion": [
                        { "$cond": [{ "$eq": ["$win", True] }, ["Windows"], []] },
                        { "$cond": [{ "$eq": ["$mac", True] }, ["Mac"], []] },
                        { "$cond": [{ "$eq": ["$linux", True] }, ["Linux"], []] },
                        { "$cond": [{ "$eq": ["$steam_deck", True] }, ["Steam Deck"], []] }
                    ]
                } 
            }
        },
        { "$unwind": "$platforms" },  # Unwind platforms to create a document for each platform
        { 
            "$group": { 
                "_id": "$platforms", 
                "average_rating": { "$avg": "$rating" }  # Calculate average rating by platform
            } 
        }
    ]
)

print("View 'average_rating_by_platform' created.")










View 'average_rating_by_platform' created.


In [31]:
##Indexes on 6 different collections that would make queries more optimal

In [42]:
#we will create an index to enhance the performance of queries that retrieve user information by user id



db.users.create_index([("user_id", 1)])


'user_id_1'

In [43]:
#Create an index which can help finding recent reviews
db.reviews.create_index([("app_id", 1), ("date", 1)])


'app_id_1_date_1'

In [44]:
#Create an index that quickly find games related with a specific tag
db.tags.create_index([("tag", 1)])






'tag_1'

In [45]:
#Create an index that will improve performance if we choose to filter games by genre and release date
db.games.create_index([("genre", 1), ("release_date", 1)])






'genre_1_release_date_1'

In [46]:
#This index will optimize searches for reviews by date range

db.reviews.create_index([("review_date", 1)])



'review_date_1'

In [47]:
#Create an index that will optimize searches for games by each  publisher
db.games.create_index([("publisher", 1)])


'publisher_1'

In [ ]:
#TESTING

In [1]:
from pymongo import MongoClient

# Connect to MongoDB 
client = MongoClient("mongodb://localhost:27017/")  
db = client['steam_games_db']  
# Count the total number of games in the games_metadata collection
total_games = db.games_metadata.count_documents({})
print(f"Total number of games: {total_games}")



Total number of games: 50872
